In [1]:
# Create SparkSession from builder
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('chapter01_to_05') \
                    .getOrCreate()

23/11/03 07:18:04 WARN Utils: Your hostname, FM-PC-LT-342 resolves to a loopback address: 127.0.1.1; using 192.168.1.74 instead (on interface wlp0s20f3)
23/11/03 07:18:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/03 07:18:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
myrange = spark.range(1000).toDF("number")

In [4]:
myrange.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



In [5]:
myrange.printSchema()

root
 |-- number: long (nullable = false)



In [6]:
divisBy2 = myrange.where("number % 2 = 0")
divisBy2.show()

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
|    20|
|    22|
|    24|
|    26|
|    28|
|    30|
|    32|
|    34|
|    36|
|    38|
+------+
only showing top 20 rows



In [7]:
divisBy2.count()

500

In [8]:
flightData2015 =  spark.read.option("inferSchema","true").\
                  option("header","true").\
                  csv("/home/fm-pc-lt-342/Documents/Spark Docx/datasets/datas/flight-data/csv/2015-summary.csv")

In [9]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [10]:
flightData2015.show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows



In [11]:
flightData2015.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [12]:
flightData2015.sort('count').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#39 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#39 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=93]
      +- FileScan csv [DEST_COUNTRY_NAME#37,ORIGIN_COUNTRY_NAME#38,count#39] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/fm-pc-lt-342/Documents/Spark Docx/datasets/datas/flight-dat..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




### Shuffle Default Partition

### By default, when we perform a shuffle, Spark
### outputs 200 shuffle partitions. Let’s set this value to 5 to reduce the number of the output
### partitions from the shuffle:

In [13]:
spark.conf.get('spark.sql.shuffle.partitions')

'200'

### Repartition to 5

In [14]:
spark.conf.set('spark.sql.shuffle.partitions',"5")

In [15]:
spark.conf.get('spark.sql.shuffle.partitions')

'5'

### DataFrames and SQL
Spark can run the same
transformations, regardless of the language, in the exact same way. You can express your
business logic in SQL or DataFrames (either in R, Python, Scala, or Java) and Spark will
compile that logic down to an underlying plan (that you can see in the explain plan) before
actually executing your code. With Spark SQL, you can register any DataFrame as a table or
view (a temporary table) and query it using pure SQL. There is no performance difference
between writing SQL queries or writing DataFrame code, they both “compile” to the same
underlying plan that we specify in DataFrame code.

In [16]:
flightData2015.sort('DEST_COUNTRY_NAME').show()
"""This code in PySpark sorts the DataFrame flightData2015 by the column named 'DEST_COUNTRY_NAME' in ascending order and then displays the first 20 rows of the resulting DataFrame
. The sort() function is used to sort one or more columns in a DataFrame, and the show() function is used to display the resulting DataFrame"""

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|             Algeria|      United States|    4|
|              Angola|      United States|   15|
|            Anguilla|      United States|   41|
| Antigua and Barbuda|      United States|  126|
|           Argentina|      United States|  180|
|               Aruba|      United States|  346|
|           Australia|      United States|  329|
|             Austria|      United States|   62|
|          Azerbaijan|      United States|   21|
|             Bahrain|      United States|   19|
|            Barbados|      United States|  154|
|             Belgium|      United States|  259|
|              Belize|      United States|  188|
|             Bermuda|      United States|  183|
|             Bolivia|      United States|   30|
|Bonaire, Sint Eus...|      United States|   58|
|              Brazil|      United States|  853|
|British Virgin Is..

"This code in PySpark sorts the DataFrame flightData2015 by the column named 'DEST_COUNTRY_NAME' in ascending order and then displays the first 20 rows of the resulting DataFrame\n. The sort() function is used to sort one or more columns in a DataFrame, and the show() function is used to display the resulting DataFrame"

#### Any dataFrame can be made into a table or view with a simple method called **createOrReplaceTempView**

In [17]:
flightData2015.createOrReplaceTempView("2015_temp_view")

we can query our data in SQL. To do so, we’ll use the spark.sql function (remember,
spark is our SparkSession variable) that conveniently returns a new DataFrame. Although this
might seem a bit circular in logic—that a SQL query against a DataFrame returns another
DataFrame—it’s actually quite powerful. This makes it possible for you to specify
transformations in the manner most convenient to you at any given point in time and not sacrifice
any efficiency to do so

In [18]:
sqlWay = spark.sql("SELECT DEST_COUNTRY_NAME, count(1) FROM 2015_temp_view GROUP BY DEST_COUNTRY_NAME")
sqlWay.show(10)

+--------------------+--------+
|   DEST_COUNTRY_NAME|count(1)|
+--------------------+--------+
|             Moldova|       1|
|             Bolivia|       1|
|             Algeria|       1|
|Turks and Caicos ...|       1|
|            Pakistan|       1|
|    Marshall Islands|       1|
|            Suriname|       1|
|              Panama|       1|
|         New Zealand|       1|
|             Liberia|       1|
+--------------------+--------+
only showing top 10 rows



### Converting to Spark Types (Literals)
Sometimes, we need to pass explicit values into Spark that are just a value (rather than a newcolumn). This might be a constant value or something we’ll need to compare to later on. The
way we do this is through literals. This is basically a translation from a given programming language’s literal value to one that Spark understands. Literals are expressions and you can use
them in the same way:

In [19]:
from pyspark.sql.functions import lit

literals = flightData2015.select("*",lit(1).alias("one")).show()

+--------------------+-------------------+-----+---+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|one|
+--------------------+-------------------+-----+---+
|       United States|            Romania|   15|  1|
|       United States|            Croatia|    1|  1|
|       United States|            Ireland|  344|  1|
|               Egypt|      United States|   15|  1|
|       United States|              India|   62|  1|
|       United States|          Singapore|    1|  1|
|       United States|            Grenada|   62|  1|
|          Costa Rica|      United States|  588|  1|
|             Senegal|      United States|   40|  1|
|             Moldova|      United States|    1|  1|
|       United States|       Sint Maarten|  325|  1|
|       United States|   Marshall Islands|   39|  1|
|              Guyana|      United States|   64|  1|
|               Malta|      United States|    1|  1|
|            Anguilla|      United States|   41|  1|
|             Bolivia|      United States|   3

Before proceeding, let’s review the fundamental concepts and definitions that we covered in Part I.
Spark is a distributed programming model in which the user specifies transformations. Multiple
transformations build up a directed acyclic graph of instructions. An action begins the process of
executing that graph of instructions, as a single job, by breaking it down into stages and tasks to
execute across the cluster. The logical structures that we manipulate with transformations and actions
are DataFrames and Datasets. To create a new DataFrame or Dataset, you call a transformation. To
start computation or convert to native language types, you call an action.

### select and selectExpr
select and selectExpr allow you to do the DataFrame equivalent of SQL queries on a table of
data:-- in SQL
SELECT * FROM dataFrameTable
SELECT columnName FROM dataFrameTable
SELECT columnName * 10, otherColumn, someOtherCol as c FROM dataFrameTable
In the simplest possible terms, you can use them to manipulate columns in your DataFrames.

In [20]:
'''
    # in Python
df.select("DEST_COUNTRY_NAME").show(2)
-- in SQL
SELECT DEST_COUNTRY_NAME FROM dfTable LIMIT 2
'''

'\n    # in Python\ndf.select("DEST_COUNTRY_NAME").show(2)\n-- in SQL\nSELECT DEST_COUNTRY_NAME FROM dfTable LIMIT 2\n'

You can select multiple columns by using the same style of query, just add more column name
strings to your select method call:

 ##### in Python
 
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

-- in SQL

SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME FROM dfTable LIMIT 2



### As discussed in “Columns and Expressions”, you can refer to columns in a number of different
ways; all you need to keep in mind is that you can use them interchangeably:

In [21]:
# in Python
from pyspark.sql.functions import expr, col, column
flightData2015.select(
expr("DEST_COUNTRY_NAME"),
col("DEST_COUNTRY_NAME"),
column("DEST_COUNTRY_NAME"))\
.show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



#### Note:
 One common error is attempting to mix Column objects and strings. For example, the followingcode will result in a compiler error:
 
df.select(col("DEST_COUNTRY_NAME"), "DEST_COUNTRY_NAME")

### Adding Columns
PySpark withColumn() is a transformation function of DataFrame which is used to change the value, convert the datatype of an existing column, create a new column, and many more. In this post, I will walk you through commonly used PySpark DataFrame column operations using withColumn() examples.

Notice that the withColumn function takes two arguments: the column name and the expression
that will create the value for that given row in the DataFrame

In [22]:
from pyspark.sql.functions import expr, col
adding_new_column = flightData2015.withColumn("new_column",expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME"))
adding_new_column.show(5)

+-----------------+-------------------+-----+----------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|new_column|
+-----------------+-------------------+-----+----------+
|    United States|            Romania|   15|     false|
|    United States|            Croatia|    1|     false|
|    United States|            Ireland|  344|     false|
|            Egypt|      United States|   15|     false|
|    United States|              India|   62|     false|
+-----------------+-------------------+-----+----------+
only showing top 5 rows



In [23]:
adding_new_col = flightData2015.withColumn("count * 2",expr("count")*2).show()

+--------------------+-------------------+-----+---------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|count * 2|
+--------------------+-------------------+-----+---------+
|       United States|            Romania|   15|       30|
|       United States|            Croatia|    1|        2|
|       United States|            Ireland|  344|      688|
|               Egypt|      United States|   15|       30|
|       United States|              India|   62|      124|
|       United States|          Singapore|    1|        2|
|       United States|            Grenada|   62|      124|
|          Costa Rica|      United States|  588|     1176|
|             Senegal|      United States|   40|       80|
|             Moldova|      United States|    1|        2|
|       United States|       Sint Maarten|  325|      650|
|       United States|   Marshall Islands|   39|       78|
|              Guyana|      United States|   64|      128|
|               Malta|      United States|    1|        

In [24]:
new_col = flightData2015.withColumn("new_column_2",col("count") * 2)
new_col.show(5)

+-----------------+-------------------+-----+------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|new_column_2|
+-----------------+-------------------+-----+------------+
|    United States|            Romania|   15|          30|
|    United States|            Croatia|    1|           2|
|    United States|            Ireland|  344|         688|
|            Egypt|      United States|   15|          30|
|    United States|              India|   62|         124|
+-----------------+-------------------+-----+------------+
only showing top 5 rows



### Renaming Columns
Though we cannot rename a column using withColumn, still we wanted to cover this as renaming is one of the common operations we perform on DataFrame. To rename an existing column use withColumnRenamed() function on DataFrame.

In [25]:
renaming_column = new_col.withColumnRenamed("new_column_2","Count * 2")
renaming_column.show(5)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Count * 2|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|       30|
|    United States|            Croatia|    1|        2|
|    United States|            Ireland|  344|      688|
|            Egypt|      United States|   15|       30|
|    United States|              India|   62|      124|
+-----------------+-------------------+-----+---------+
only showing top 5 rows



### Case Sensitivity
By default Spark is case insensitive; however, you can make Spark case sensitive by setting the configuration:

-- in SQL

set spark.sql.caseSensitive true

### Removing Columns
df.drop("ORIGIN_COUNTRY_NAME")

We can drop multiple columns by passing in multiple columns as arguments:

df.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME")

In [26]:
drop_column = renaming_column.drop(col("`Count * 2`"))
drop_column.show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows



### Changing a Column’s Type (cast)

In [27]:
drop_column.withColumn("count",col("count").cast("long")).printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



### Filtering Rows
To filter rows, we create an expression that evaluates to true or false. There are two methods to perform this operation: we can use where or filter
and they both will perform the same operation and accept the same argument types when used
with DataFrames.

In [28]:
flightData2015.filter(flightData2015['count']<2).show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 3 rows



In [29]:
flightData2015.filter(col("count")<3).show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 3 rows



In [30]:
flightData2015.where(col("count")>5).show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
+-----------------+-------------------+-----+
only showing top 3 rows



### 
Instinctually, you might want to put multiple filters into the same expression. Although this is possible, it is not always useful, because Spark automatically performs all filtering operations at
the same time regardless of the filter ordering. This means that if you want to specify multiple
AND filters, just chain them sequentially and let Spark handle the rest:

In [34]:
flightData2015.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [35]:
flightData2015.filter(col("count") < 2).filter(col("ORIGIN_COUNTRY_NAME") != "Croatia").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



### Getting Unique Rows

A very common use case is to extract the unique or distinct values in a DataFrame. These values
can be in one or more columns. The way we do this is by using the distinct method on a
DataFrame, which allows us to deduplicate any rows that are in that DataFrame. For instance,
let’s get the unique origins in our dataset. This, of course, is a transformation that will return a
new DataFrame with only unique rows

In [38]:
flightData2015.select(col("DEST_COUNTRY_NAME"),col("ORIGIN_COUNTRY_NAME")).distinct().count()

256

In [24]:
flightData2015.select("DEST_COUNTRY_NAME","ORIGIN_COUNTRY_NAME").distinct().count()

256

### Sorting Rows

When we sort the values in a DataFrame, we always want to sort with either the largest or
smallest values at the top of a DataFrame. There are two equivalent operations to do this sort
and orderBy that work the exact same way. They accept both column expressions and strings as
well as multiple columns. The default is to sort in ascending orde

In [42]:
# flightData2015.sort("count","DEST_COUNTRY_NAME").show(5)
# flightData2015.orderBy("count", "DEST_COUNTRY_NAME").show(5)
flightData2015.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [43]:
# To more explicitly specify sort direction, you need to use the asc and desc functions if operating
# on a column. These allow you to specify the order in which a given column should be sorted:
from pyspark.sql.functions import desc, asc

flightData2015.orderBy(expr("count desc")).show(2)
flightData2015.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

# An advanced tip is to use asc_nulls_first, desc_nulls_first, asc_nulls_last, or
# desc_nulls_last to specify where you would like your null values to appear in an ordered
# DataFrame.

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows



### Limit
Oftentimes, you might want to restrict what you extract from a DataFrame; for example, you
might want just the top ten of some DataFrame. You can do this by using the limit method.

In [46]:
flightData2015.limit(11).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
|    United States|       Sint Maarten|  325|
+-----------------+-------------------+-----+



In [47]:
flightData2015.rdd.getNumPartitions() # 1

1

In [28]:
flightData2015.repartition(5)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

### Collect, Take and Show

collect() : collect gets all data from the entire DataFrame.

take(N) : take gets the first N rows.

show() :  prints out a number of rows nicely.

In [48]:
flightData2015.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Sint Maarten', count=325),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Marshall Islands', count=39),
 

In [49]:
flightData2015.take(10)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [50]:
flightData2015.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri